In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-31 18:24:19,650 INFO: Initializing external client


2025-01-31 18:24:19,655 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-01-31 18:24:21,709 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [4]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [5]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [6]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [7]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.15s) 


'2025-01-28 00:00:00 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [8]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [9]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  b8642d5f958e6fe3ba79455bbb96bf2035f7898a  -  2025-01-30 23:49:00+00:00


Processing commit:  7ae0b63c218807f65d97228ad20c15b62710bfb5  -  2025-01-30 22:50:17+00:00


Processing commit:  6f0abfd68af8e023aa534aab07e8b00791cb60bc  -  2025-01-30 21:48:59+00:00


Processing commit:  63e73056184244174da7b364d231d1e9f18297b6  -  2025-01-30 20:49:10+00:00


Processing commit:  74f784432c6656e28af3bd385d4a6428fc5f4374  -  2025-01-30 19:49:04+00:00


...

Processing commit:  e81055b5d817c13a07b84dcc96a03b0c4cf339f4  -  2025-01-28 03:49:04+00:00


Processing commit:  fa9c461caa401783242b5e4f66a8196b404d433c  -  2025-01-28 02:56:18+00:00


Processing commit:  4b590d9954609f82f0d542a2e1131271e2ff5052  -  2025-01-28 01:33:32+00:00


Processing commit:  48871f75d734ae002e5a14fa10ac7880001ee5e4  -  2025-01-28 00:49:43+00:00


breaking at:  2025-01-27 23:49:07+00:00
{'CLARENDON ROW': [[14, datetime.datetime(2025, 1, 30, 23, 49, tzinfo=<git.objects.util.tzoffset object at 0x0000022B997963E0>)], [15, datetime.datetime(2025, 1, 30, 22, 50, 17, tzinfo=<git.objects.util.tzoffset object at 0x0000022B99796AA0>)], [20, datetime.datetime(2025, 1, 30, 21, 48, 59, tzinfo=<git.objects.util.tzoffset object at 0x0000022B99796410>)], [20, datetime.datetime(2025, 1, 30, 20, 49, 10, tzinfo=<git.objects.util.tzoffset object at 0x0000022B99796290>)], [20, datetime.datetime(2025, 1, 30, 19, 49, 4, tzinfo=<git.objects.util.tzoffset object at 0x0000022B997B64A0>)], [23, datetime.datetime(2025, 1, 30, 18, 49, 8, tzinfo=<git.objects.util.tzoffset object at 0x0000022B997BBA60>)], [14, datetime.datetime(2025, 1, 30, 17, 48, 59, tzinfo=<git.objects.util.tzoffset object at 0x0000022B998F9060>)], [19, datetime.datetime(2025, 1, 30, 16, 50, 3, tzinfo=<git.objects.util.tzoffset object at 0x0000022B99902620>)], [22, datetime.datetime(2025,

In [10]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today['datetime'] = pd.to_datetime(df_bike_today['datetime'], utc=True).dt.floor('H') + datetime.timedelta(hours=1)
df_bike_today


,num_bikes_available,datetime,station
0,37.0,2025-01-31 00:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
1,37.0,2025-01-30 23:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
2,38.0,2025-01-30 22:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
3,40.0,2025-01-30 21:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
4,40.0,2025-01-30 20:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
...,...,...,...
67,40.0,2025-01-28 05:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
68,40.0,2025-01-28 04:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
69,40.0,2025-01-28 03:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
70,40.0,2025-01-28 02:00:00+00:00,HEUSTON_BRIDGE_(NORTH)


## Fetch the weather data for the same time period

In [11]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}


Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-31 00:00:00+00:00,6.95,2.110960,0.4,0.0,25.929996,32065.236328,0.400000,dublin
1,2025-01-31 01:00:00+00:00,6.75,1.755430,0.0,0.0,26.932714,32065.236328,0.400000,dublin
2,2025-01-31 02:00:00+00:00,6.35,1.483531,0.0,0.0,25.233406,32065.236328,0.400000,dublin
3,2025-01-31 03:00:00+00:00,5.90,1.232717,0.0,0.0,23.224882,32065.236328,0.400000,dublin
4,2025-01-31 04:00:00+00:00,5.45,0.968557,0.0,0.0,21.325253,32065.236328,0.400000,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-02-09 19:00:00+00:00,5.20,0.061158,0.1,0.0,23.806318,34126.535156,5.699998,dublin
236,2025-02-09 20:00:00+00:00,5.20,-0.020061,0.1,0.0,24.363251,34126.535156,5.699998,dublin
237,2025-02-09 21:00:00+00:00,5.20,-0.084013,0.1,0.0,24.590923,34126.535156,5.699998,dublin
238,2025-02-09 22:00:00+00:00,5.15,-0.177298,0.1,0.0,24.826952,34126.535156,5.699998,dublin


## Insert the bike and weather data into Hopsworks

In [12]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 0.00% |                                      | Rows 0/72 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 1.39% |▍                                 | Rows 1/72 | Elapsed Time: 00:01 | Remaining Time: 01:19

Uploading Dataframe: 100.00% |███████████████████████████████| Rows 72/72 | Elapsed Time: 00:01 | Remaining Time: 00:00

Launching job: bike_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [13]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 0.00% |                                     | Rows 0/240 | Elapsed Time: 00:00 | Remaining Time: ?

Uploading Dataframe: 69.17% |████████████████████▊         | Rows 166/240 | Elapsed Time: 00:01 | Remaining Time: 00:00

Uploading Dataframe: 100.00% |█████████████████████████████| Rows 240/240 | Elapsed Time: 00:01 | Remaining Time: 00:00

Launching job: weather_data_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions


2025-01-31 18:25:51,023 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED


2025-01-31 18:26:16,584 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED


2025-01-31 18:28:43,413 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED


2025-01-31 18:28:43,579 INFO: Waiting for log aggregation to finish.


2025-01-31 18:29:02,193 INFO: Execution finished successfully.
